<a href="https://colab.research.google.com/github/Shelfy22/test/blob/main/Building_Agentic_RAG_Llamaindex_1_Multi_Document_Agent_04_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Концепты использования RAG (Retrieval-Augmented Generation)
## Создание агентского RAG с Llamaindex

По мативам курса "Building Agentic RAG with Llamaindex" https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/1/introduction

Документация: https://docs.llamaindex.ai/en/stable/

Приглашаю в Телеграм общаться по этой теме: https://t.me/AiExp01

## Введение

В этом курсе вы узнаете о фреймворке Agentic RAG Llamaindex для создания исследовательских агентов, способных использовать передовые инструменты, рассуждать и принимать решения с вашими данными.

Этот курс покажет, как построить автономного исследовательского агента, повышая уровень взаимодействия с вашими данными.


## Установка библиотек, вспомогательные функции

In [ ]:
!pip install llama-index==0.10.59
!pip install llama-index-llms-openai==0.1.27
!pip install llama-index-embeddings-openai==0.1.5
!pip install nest-asyncio==1.6.0

INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Устанавливаем API ключ
from google.colab import userdata
OPEN_AI_API_KEY = userdata.get('OpenAI_API_KEY')
import os
os.environ["OPENAI_API_KEY"] = OPEN_AI_API_KEY
LL_MODEL='gpt-4o'
# LL_MODEL='gpt-3.5-turbo'

## Создание Агентов для работы с несколькими документами

 В этом уроке мы расширим возможности агента, чтобы он мог обрабатывать несколько документов.
 Настроим нашего агента для обработки сразу трех книг.

In [ ]:
from typing import List, Optional
from pathlib import Path
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import FunctionTool
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core import SummaryIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings

import nest_asyncio
nest_asyncio.apply()

def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Получает инструменты для векторного поиска и суммаризации из документа."""

    # загрузка документов из указанного пути
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    # разбиение документов на чанки, с максимальным размером чанка 1024
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)

    # создание индекса для векторного поиска по чанкам
    vector_index = VectorStoreIndex(nodes)

    def vector_query(
        query: str,
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Используется для ответа на вопросы по данному документу.

        Полезно, если у вас есть конкретные вопросы по документу.
        Всегда оставляйте page_numbers как None, ЕСЛИ ТОЛЬКО у вас нет конкретных страниц, которые вы хотите искать.

        Args:
            query (str): строка запроса, которая будет преобразована в вектор.
            page_numbers (Optional[List[str]]): Фильтрация по набору страниц. Оставьте как NONE
                если мы хотим выполнить векторный поиск
                по всем страницам. В противном случае, фильтрация по указанному набору страниц.

        """

        # Если страницы не указаны, используем пустой список
        page_numbers = page_numbers or []

        # Создание списка словарей метаданных для фильтрации по номерам страниц
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        # Настройка поискового движка с фильтрами по метаданным
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )

        # Выполнение запроса и возврат результата
        response = query_engine.query(query)
        return response

    # Создание инструментов для векторного запроса с заданными параметрами
    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )

    # Создание индекса для саммаризации
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )

    # Создание инструмента для суммаризации с заданными параметрами
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            f"Useful for summarization questions related to {name}"
        ),
    )

    # Возвращаем инструменты векторного поиска и суммаризации
    return vector_query_tool, summary_tool

In [ ]:
# Список наших книг:
papers = [
    "Architecture_30_seconds.pdf",
    "Chemistry_30_seconds.pdf",
    "Math_30_seconds.pdf",
]

In [ ]:
# Объединяем vector_tool и summary_tool в список и передаем его агенту, так что у агента в итоге будет 6 инструментов.
paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: Architecture_30_seconds.pdf
Getting tools for paper: Chemistry_30_seconds.pdf
Getting tools for paper: Math_30_seconds.pdf


In [ ]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]
from llama_index.llms.openai import OpenAI
llm = OpenAI(model=LL_MODEL)
len(initial_tools)

6

In [ ]:
# Cоздаем agent_worker.Этот агент включает в себя 6 инструментов и языковую модель
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
# Теперь мы можем задавать вопросы, касающиеся этих 3 документов, или одного из них.
response = agent.query(
    "Есть ли в книге ифнормация о Ферма?"
    "Если есть,то в какой книге?"
)

Added user message to memory: Есть ли в книге ифнормация о Ферма?Если есть,то в какой книге?
=== Calling Function ===
Calling function: vector_tool_Math_30_seconds with args: {"query": "\u0424\u0435\u0440\u043c\u0430"}
=== Function Output ===
Fermat was a mathematician known for his work in geometry, probability theory, physics, and number theory. He famously formulated Fermat's Last Theorem, which remained unsolved for 300 years. Despite his significant contributions to mathematics and his reputation as the founder of modern number theory, Fermat preferred to maintain his status as an amateur mathematician throughout his life. He often discussed his ideas and discoveries through letters and manuscripts, avoiding formal publications. Fermat's reluctance to share his proofs with the wider public led some of his colleagues to speculate that he may not have had proofs for some of his claims.
=== Calling Function ===
Calling function: vector_tool_Chemistry_30_seconds with args: {"query": "

In [ ]:
# Теперь мы можем задавать вопросы, касающиеся этих 3 документов, или одного из них.
response = agent.query(
    "Рааскажи про теорему Ферма, дай краткое описание для новичков. Каждое предложение начинай с новой строки"
)

Added user message to memory: Рааскажи про теорему Ферма, дай краткое описание для новичков. Каждое предложение начинай с новой строки
=== Calling Function ===
Calling function: vector_tool_Math_30_seconds with args: {"query": "\u0422\u0435\u043e\u0440\u0435\u043c\u0430 \u0424\u0435\u0440\u043c\u0430"}
=== Function Output ===
Fermat's Last Theorem was a famous mathematical problem that remained unsolved for 300 years. Fermat, a mathematician known for his contributions to geometry, probability theory, and number theory, left a note in one of his books with the theorem, which added to the mystery surrounding it. Despite his significant impact on mathematics and science, Fermat chose to maintain his status as an amateur mathematician throughout his life, sharing his ideas and discoveries through letters and manuscripts rather than formal publications.
=== LLM Response ===
Теорема Ферма была известной математической проблемой, которая оставалась нерешенной в течение 300 лет.  
Ферма, мате

In [ ]:
# Теперь мы можем задавать вопросы, касающиеся этих 3 документов, или одного из них.
response = agent.query(
    "Что такое периодическая таблица?"
    "Кто автор?"
    "В какой книге информация об этом?"
)

Added user message to memory: Что такое периодическая таблица?Кто автор?В какой книге информация об этом?
=== Calling Function ===
Calling function: vector_tool_Chemistry_30_seconds with args: {"query": "\u0427\u0442\u043e \u0442\u0430\u043a\u043e\u0435 \u043f\u0435\u0440\u0438\u043e\u0434\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u0442\u0430\u0431\u043b\u0438\u0446\u0430?"}
=== Function Output ===
Периодическая таблица - это систематическое упорядочение всех известных химических элементов в порядке возрастания атомного номера, где элементы схожи по химическим и физическим свойствам с элементами, стоящими рядом с ними.
=== Calling Function ===
Calling function: vector_tool_Chemistry_30_seconds with args: {"query": "\u041a\u0442\u043e \u0430\u0432\u0442\u043e\u0440 \u043f\u0435\u0440\u0438\u043e\u0434\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u0442\u0430\u0431\u043b\u0438\u0446\u044b?"}
=== Function Output ===
Дмитрий Иванович Менделеев.
=== LLM Response ===
Периодическая таблица — эт

In [ ]:
# Теперь мы можем задавать вопросы, касающиеся этих 3 документов, или одного из них.
response = agent.query(
    "Что такое классицизм?"
    "В какой книге информация об этом?"
)

Added user message to memory: Что такое классицизм?В какой книге информация об этом?
=== Calling Function ===
Calling function: vector_tool_Architecture_30_seconds with args: {"query": "\u043a\u043b\u0430\u0441\u0441\u0438\u0446\u0438\u0437\u043c"}
=== Function Output ===
Classicism refers to the use of ancient principles and models, typically to symbolically embody ideas of order and harmony. It has been established as an architectural style for state and civic institutions, such as museums, banks, and train stations. In the 19th century, notable structures were erected in this style by architects like Karl Friedrich Schinkel in Prussia and the architectural firm McKim, Mead & White in the United States. Classicism has evolved through various phases like Mannerism, Baroque, Rococo, Palladianism, Neoclassicism, Greek Revival, and Beaux-Arts, each inheriting or interpreting ancient Greek and Roman models.
=== Calling Function ===
Calling function: vector_tool_Chemistry_30_seconds with a

#Внимание:

1. Если вы испытываете проблемы с просмотром ранее опубликованного видео в связи с замедлением Youtube, пишите в кометы под нужным постом в Телеграм - и я залью туда это видео.
2. Приглашаю в Телеграм общаться по этой теме: https://t.me/AiExp01
3. Чат находится здесь: https://t.me/AiExp02
4. Что еще изучить и посмотреть:

*   LlamaIndex https://www.youtube.com/watch?v=myGdYwP83D8&list=PLAzpexGM7JahXip0jNyDEJwPj-wySj3Ix
*   Концепты использования OpenAI API (API ChatGPT)
Playlist: https://www.youtube.com/watch?v=FoLquyNFE_c&list=PLAzpexGM7JagNMDIWUsoO8fbTwbY82ZEN
*   Концепты использования RAG (Retrieval-Augmented Generation)
Playlist: https://www.youtube.com/watch?v=NkjkqsLCweQ&list=PLAzpexGM7Jai1qgfxMkY-6ivOft-7dh8Q
